In [3]:
import pandas as pd
sample=pd.read_csv('busines_norm.csv')

In [4]:
sample = sample[sample.norm!='-'].dropna()
y = sample.norm
sample = sample.drop(['Unnamed: 0', 'norm', 'text'], axis=1)

In [5]:
import re
i=0;
for sent in sample['title'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sno = nltk.stem.SnowballStemmer('russian') #initialising the snowball stemmer which is developed in recent years
stop=set(stopwords.words('russian'))



def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

[nltk_data] Downloading package stopwords to /root/nltk_data...


{'нет', 'всего', 'между', 'вы', 'хорошо', 'этот', 'мне', 'перед', 'сейчас', 'них', 'их', 'чтоб', 'есть', 'раз', 'были', 'можно', 'конечно', 'же', 'когда', 'от', 'во', 'себе', 'вам', 'того', 'свою', 'где', 'него', 'ж', 'ведь', 'не', 'всех', 'даже', 'под', 'по', 'ни', 'нас', 'этом', 'себя', 'такой', 'никогда', 'хоть', 'ты', 'про', 'у', 'до', 'с', 'из', 'там', 'для', 'они', 'я', 'он', 'чуть', 'тоже', 'зачем', 'наконец', 'что', 'ну', 'был', 'всегда', 'куда', 'ли', 'только', 'еще', 'разве', 'потому', 'моя', 'более', 'этого', 'им', 'об', 'да', 'три', 'лучше', 'его', 'том', 'больше', 'чего', 'надо', 'над', 'опять', 'за', 'без', 'было', 'а', 'будет', 'ей', 'здесь', 'ней', 'чтобы', 'какой', 'вас', 'совсем', 'о', 'мы', 'ему', 'к', 'много', 'почти', 'тебя', 'нее', 'чем', 'иногда', 'или', 'кто', 'нибудь', 'впрочем', 'быть', 'через', 'тогда', 'меня', 'будто', 'другой', 'потом', 'нельзя', 'эти', 'может', 'то', 'сам', 'уже', 'так', 'бы', 'в', 'после', 'теперь', 'ним', 'мой', 'но', 'все', 'при', 'вот'

[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in sample['title'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (sample['title'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(sample['title'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [8]:
sample['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
sample['CleanedText']=sample['CleanedText'].str.decode("utf-8")

In [9]:
i=0
list_of_sent_train=[]
for sent in sample['CleanedText'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent_train.append(filtered_sentence)

In [10]:
import gensim
# Training the wor2vec model using train dataset
w2v_model=gensim.models.Word2Vec(list_of_sent_train,workers=4)

In [11]:
import numpy as np
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this train
for sent in list_of_sent_train: # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)
sent_vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(466, 100)

In [12]:
x = sent_vectors

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=4)

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
clf=DecisionTreeClassifier()
parametrs = {'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': range(2,5), 'min_samples_split': range(1,5), 'min_samples_leaf': range(1,5)}
grid_clf = GridSearchCV(clf, parametrs, cv = 5)
grid_clf.fit(x,y)
grid_clf.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
360 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 254, in fit
    % self.min_samples_split
ValueError: min_samples_split must be an integer greater than 1 or a floa

{'criterion': 'entropy',
 'max_depth': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [43]:
clf=DecisionTreeClassifier(criterion='entropy', max_depth=2,min_samples_leaf=1,min_samples_split=2)
clf.fit(x_train, y_train)
from sklearn.model_selection import cross_validate
def cross_metrics(model, X_train, y_train):
    scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted']
    scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=3)
    sorted(scores.keys())
    LR_fit_time = scores['fit_time'].mean()
    LR_score_time = scores['score_time'].mean()
    LR_accuracy = scores['test_accuracy'].mean()
    LR_precision = scores['test_precision_macro'].mean()
    LR_recall = scores['test_recall_macro'].mean()
    LR_f1 = scores['test_f1_weighted'].mean()
    
    print("fit_time: ",LR_fit_time)
    print("score time: ",LR_score_time)
    print("test_accuracy: ",LR_accuracy)
    print("test_precision_macro: ",LR_precision)
    print("test_recall_macro: ",LR_recall)
    print("test_f1_weighted: ",LR_f1)
cross_metrics(clf, x, y)

fit_time:  0.01503300666809082
score time:  0.004897117614746094
test_accuracy:  0.4419768403639372
test_precision_macro:  0.17890200785831714
test_recall_macro:  0.24023788102735474
test_f1_weighted:  0.32846961602406016


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
